<div style="background-image: url('style_notebook.jpeg'); padding: 20px;
            border-radius: 10px; background-attachment: fixed;">
    <div style="margin: 20px; margin-top: 100px; color: white;">
        <center>
            <b style="font-size: 40px;">Network Analysis: FInal Project</b>
            <br>
            <br>
            <b style="font-size: 30px;">Preprocess</b>
            <br>
        </center>
        <div style="font-size: 20px; margin-left: 60%; margin-top: 50px;">
            <b>Trabalho reallizado por:</b>
            <ul>
                <li>Rodrigo Sarroeira, nº 92761</li>
                <li>Wendel Vilaça, nº 109881</li>
            </ul>
        </div>
    </div>
</div>

In [14]:
# This means that all the needed databases are ready to use
saved = False

<div style="background-image: url('style_notebook.jpeg'); padding: 10px;
            border-radius: 20px; color: white; background-attachment: fixed;">
    <center><h1>Introduction</h1></center>
</div>

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">In this first section of our study, we present the case-study and prepare the data for analysis and modeling. The database in use contains 1458644 observations relative to taxi trips in New York City.This database includes the collection time, geographic coordinates, number of passengers and several other variables.
<br /><br />
The main goal of this notebook is to prepare the data to be used for network analysis. For each trip, the database contains the coordinates of the pickup and the dropout locations. This information is not very usefull to import to a network, because the coordinates are continuous variables. This would be a problem, because almost every trip would generate two new nodes in the graph, created by the coordinates of the dropout and pickup locations. To overcome this problem, a data transformation method is implemented. 
<br /><br />
The transformation consists in using the coordinates to assign each trip to a pickup area and a droupout area. This will allow analysing the data through a network analysis point of view. Each node will correspond to a certain area of NYC, and each connection represents a trip between these two areas. 


<div style="background-image: url('style_notebook.jpeg'); padding: 10px;
            border-radius: 20px; color: white; background-attachment: fixed;">
    <center><h1>0. Imports</h1></center>
</div>

In [15]:
import json
import random
import folium
import pickle
import geopandas
import numpy as np
import pandas as pd
import networkx as nx
from folium import plugins
import shapely.geometry as geo
import matplotlib.pyplot as plt
from folium.plugins import HeatMap

<div style="background-image: url('style_notebook.jpeg'); padding: 10px;
            border-radius: 20px; color: white; background-attachment: fixed;">
    <center><h1>1. Read Data</h1></center>
</div>

In [16]:
data = pd.read_csv("data/csv/data.csv")

data.rename(columns={"pickup_latitude": "x1",
                     "pickup_longitude": "y1",
                     "dropoff_latitude": "x2",
                     "dropoff_longitude": "y2",
                     "pickup_datetime": "time1",
                     "dropoff_datetime": "time2",
                     "passenger_count": "n_pass"}, inplace=True)

In [17]:
len(data)

1458644

In [18]:
random.seed(1)
data = data.sample(int(len(data) * 0.20))
len(data)

291728

In [19]:
# The database does not contain missing values
len(data) == len(data.dropna())

True

In [20]:
data

,id,vendor_id,time1,time2,n_pass,y1,x1,y2,x2,store_and_fwd_flag,trip_duration
1187786,id1456301,2,2016-06-06 07:45:52,2016-06-06 08:02:07,1,-73.935379,40.850029,-73.919487,40.836658,N,975
269282,id0044196,1,2016-02-21 17:46:17,2016-02-21 18:02:19,1,-73.951378,40.774197,-73.986221,40.777676,N,962
785115,id0337671,1,2016-02-11 13:54:27,2016-02-11 13:59:47,1,-73.964005,40.771309,-73.968765,40.764549,N,320
683093,id3554763,2,2016-05-25 13:17:14,2016-05-25 13:32:22,5,-73.975601,40.750450,-73.992523,40.749699,N,908
1159448,id3361008,2,2016-05-17 20:52:20,2016-05-17 21:05:15,3,-73.957367,40.768768,-73.947617,40.801170,N,775
...,...,...,...,...,...,...,...,...,...,...,...
589205,id0763794,1,2016-01-08 19:21:12,2016-01-08 19:26:53,1,-73.968529,40.759624,-73.954819,40.773178,N,341
957752,id0306105,1,2016-04-20 12:47:26,2016-04-20 12:56:35,1,-73.980644,40.782269,-73.980980,40.774109,N,549
476469,id0580539,1,2016-04-27 15:00:12,2016-04-27 15:09:57,1,-73.973572,40.743542,-74.006042,40.706375,N,585
437619,id0052221,2,2016-06-10 01:25:04,2016-06-10 01:33:38,1,-74.001343,40.727798,-73.986389,40.741798,N,514


<div style="background-image: url('style_notebook.jpeg'); padding: 10px;
            border-radius: 20px; color: white; background-attachment: fixed;">
    <center><h1>2. Data Understanding</h1></center>
</div>

<div style="border: 4px solid black;
            border-radius: 20px; color: white; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>2.1 Variable Description</h1></center>
</div>

<ul style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">
<li><b>id</b> - a unique identifier for each trip</li>
<li><b>vendor_id</b> - a code indicating the provider associated with the trip record</li>
<li><b>time1</b> - date and time when the meter was engaged</li>
<li><b>time2</b> - date and time when the meter was disengaged</li>
<li><b>n_pass</b> - the number of passengers in the vehicle (driver entered value)</li>
<li><b>pickup_longitude</b> - the longitude where the meter was engaged</li>
<li><b>pickup_latitude</b> - the latitude where the meter was engaged</li>
<li><b>dropoff_longitude</b> - the longitude where the meter was disengaged</li>
<li><b>dropoff_latitude</b> - the latitude where the meter was disengaged</li>
<li><b>store_and_fwd_flag</b> - Indicates whether the trip record was held in vehicle memory</li>
<li><b>trip_duration</b> - duration of the trip in seconds</li>
</ul>

<div style="background-image: url('style_notebook.jpeg'); padding: 10px;
            border-radius: 20px; color: white; background-attachment: fixed;">
    <center><h1>3. Data Transformation</h1></center>
</div>

<div style="border: 4px solid black;
            border-radius: 20px; color: white; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>3.1 Filter observations that are inside NYC</h1></center>
</div>

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">
    Firstly, the trips are filtered by their coordinates. Only observations near NYC are kept. Some of the deleted observations are errors or trips with long locations. For this study the goal is to study only the trips in NYC and its surroundings. After applying the filters bellow, 10.41% of the data is deleted.
</p>

In [21]:
# Filter pickups by latitude
data = data[(40.683280 < data["x1"]) & (data["x1"] < 40.846102)]

In [22]:
# Filter pickups by longitude
data = data[(-74.030592 < data["y1"]) & (data["y1"] < -73.892724)]

In [23]:
# Filter dropout by latitude
data = data[(40.683280 < data["x2"]) & (data["x2"] < 40.846102)]

In [24]:
# Filter dropout by longitude
data = data[(-74.030592 < data["y2"]) & (data["y2"] < -73.892724)]

In [26]:
len(data) / 291728 - 1

-0.10407297208358468

<div style="border: 4px solid black;
            border-radius: 20px; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>3.2 Create distance column</h1></center>
</div>

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">
The problem proposed in this mathematical work requires the need to use a solution by the Euclidean distance, that is, it is the distance between points, which can be proved by applying the Pythagorean repetition. Applying this formula as distance, Euclidean space becomes a metric space.</p>
<br>
<img src="euc.jpg">

In [118]:
euc_dist = ((data["x1"] - data["x2"])**2 + (data["y1"] - data["y2"])**2) ** (1/2)
data["dist"] = euc_dist

<div style="border: 4px solid black;
            border-radius: 20px; color: white; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>3.3 Create influence areas</h1></center>
</div>

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">
    As said above each set of coordinates will be assigned to a determined influence area. To acomplish this, a grid of equily-distant points over NYC is computed. Each point in the grid represent an influence area, 270 are created. The gap between areas is 0.02º, representing XXXm, aproximatelly.
    </p>

In [28]:
# Create coord ranges over the desired raster extension
step = 0.01
lngs = np.arange(-74.030592, -73.892724 + step, step)
lats = np.arange(40.683280, 40.846102 + step, step)

In [29]:
# Create grid of points
points = [(lat, lng) for lat in lats for lng in lngs]
lat, lng = [p[0] for p in points], [p[1] for p in points]
area_id = range(len(points))

In [30]:
# Each area will be represented by and id, latitude and longitude
areas = pd.DataFrame({"area_id": area_id, "lat": lat, "lng": lng})
areas.head(5)

,area_id,lat,lng
0,0,40.68328,-74.030592
1,1,40.68328,-74.020592
2,2,40.68328,-74.010592
3,3,40.68328,-74.000592
4,4,40.68328,-73.990592


In [31]:
if not saved:
    pickle.dump(areas, open("data/nx/areas_df", "wb"))

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">
    The following map presents the computed influence areas and 1000 trips represented. The red marker represents the pickup location, and the green marker the dropout location. </p>

In [32]:
m = folium.Map(location=[40.770610, -73.950242], zoom_start=11, control_scale=True)

for i, area in areas.iterrows():
    
    folium.CircleMarker(location=(area["lat"], area["lng"]), radius=1, color='black', opacity=1,
                        popup=f"{area['area_id']}: {area['lat']}, {area['lng']}").add_to(m)
    
for i, point in data.iloc[0:100].iterrows():
    
    folium.CircleMarker(location=(point["x1"], point["y1"]), radius=1, opacity=1,
                        color='red', popup=f"Start: {str(point['id'])}").add_to(m)
    
    folium.CircleMarker(location=(point["x2"], point["y2"]), radius=1, opacity=1,
                        color='green', popup=f"End: {str(point['id'])}").add_to(m)

m

<div style="border: 4px solid black;
            border-radius: 20px; color: white; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>3.4 Assign each trip to a given influence area</h1></center>
</div>

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">
    This following piece of code is used to assign each set of coordiates to a determined influence area. To acomplish these, for each trip, the distance between two sets of coordinates and all the influence areas must be computed, to find the closest one. The database contains almost 300Th observations and 270 influence areas are developed, all the euclidean distances between this points and areas must be computed. Each eucledean distance perfoms 6 operations (2 subtractions, two squares, one sum, and on square root). This represents an aproximate total of 291 728 * 270 * 2 * 6 = 997 709 760 aritmetic operations. This algorith has an high time complexity.
    </p>

In [124]:
if not saved:
    
    areas1 = []
    areas2 = []

    data = data.reset_index()
    l = len(data)

    for idx1, point in data.iterrows():
        areas_copy = areas.copy()
        distances1 = []
        distances2 = []
        for _, area in areas.iterrows():
            p1 = np.array([point["x1"], point["y1"]])
            p2 =  np.array([point["x2"], point["y2"]])
            a = np.array([area["lat"], area["lng"]])
            distances1.append(np.linalg.norm(p1 - a))
            distances2.append(np.linalg.norm(p2 - a))
        areas1.append(np.argmin(distances1))
        areas2.append(np.argmin(distances2))
        if idx1 % 100 == 0:
            print(idx1/l*100)

    data["area1"] = areas1
    data["area2"] = areas2

0.0
0.03827384939240264
0.07654769878480527
0.11482154817720792
0.15309539756961055
0.1913692469620132
0.22964309635441585
0.26791694574681846
0.3061907951392211
0.3444646445316238
0.3827384939240264
0.42101234331642906
0.4592861927088317
0.49756004210123433
0.5358338914936369
0.5741077408860396
0.6123815902784422
0.6506554396708448
0.6889292890632476
0.7272031384556502
0.7654769878480528
0.8037508372404554
0.8420246866328581
0.8802985360252606
0.9185723854176634
0.9568462348100661
0.9951200842024687
1.0333939335948714
1.0716677829872738
1.1099416323796767
1.148215481772079
1.186489331164482
1.2247631805568844
1.2630370299492872
1.3013108793416897
1.3395847287340925
1.3778585781264951
1.4161324275188978
1.4544062769113004
1.492680126303703
1.5309539756961057
1.5692278250885083
1.6075016744809107
1.6457755238733136
1.6840493732657162
1.7223232226581189
1.7605970720505213
1.798870921442924
1.8371447708353268
1.8754186202277294
1.9136924696201323
1.9519663190125345
1.9902401684049373
2.02

16.95531528083437
16.993589130226773
17.031862979619174
17.070136829011577
17.108410678403978
17.14668452779638
17.184958377188785
17.22323222658119
17.26150607597359
17.299779925365993
17.338053774758396
17.3763276241508
17.4146014735432
17.452875322935604
17.491149172328008
17.529423021720408
17.567696871112812
17.605970720505216
17.64424456989762
17.68251841929002
17.720792268682423
17.759066118074827
17.797339967467227
17.83561381685963
17.873887666252035
17.91216151564444
17.95043536503684
17.988709214429242
18.026983063821646
18.065256913214046
18.10353076260645
18.14180461199885
18.180078461391254
18.218352310783654
18.256626160176058
18.29490000956846
18.333173858960865
18.371447708353266
18.40972155774567
18.447995407138073
18.486269256530473
18.524543105922877
18.56281695531528
18.601090804707685
18.639364654100085
18.67763850349249
18.715912352884892
18.754186202277296
18.792460051669696
18.8307339010621
18.869007750454504
18.907281599846904
18.945555449239308
18.98382929863

33.872356712276336
33.91063056166874
33.94890441106114
33.98717826045355
34.02545210984595
34.06372595923835
34.10199980863075
34.140273658023155
34.17854750741556
34.216821356807955
34.255095206200366
34.29336905559276
34.33164290498517
34.36991675437757
34.408190603769974
34.44646445316238
34.48473830255478
34.52301215194718
34.56128600133959
34.599559850731985
34.63783370012439
34.67610754951679
34.7143813989092
34.7526552483016
34.790929097694004
34.8292029470864
34.867476796478805
34.90575064587121
34.94402449526361
34.982298344656016
35.02057219404841
35.058846043440816
35.09711989283322
35.135393742225624
35.17366759161803
35.21194144101043
35.25021529040283
35.28848913979524
35.326762989187635
35.36503683858004
35.40331068797244
35.44158453736485
35.47985838675724
35.518132236149654
35.55640608554205
35.594679934934454
35.63295378432686
35.67122763371926
35.709501483111666
35.74777533250407
35.786049181896466
35.82432303128888
35.862596880681274
35.90087073007368
35.93914457946

51.13386278824993
51.17213663764233
51.21041048703473
51.24868433642714
51.28695818581954
51.32523203521194
51.36350588460434
51.40177973399675
51.440053583389144
51.478327432781555
51.51660128217396
51.554875131566355
51.59314898095876
51.631422830351156
51.66969667974356
51.70797052913597
51.746244378528374
51.78451822792077
51.822792077313174
51.86106592670557
51.89933977609799
51.937613625490386
51.97588747488279
52.014161324275186
52.05243517366759
52.090709023059986
52.1289828724524
52.1672567218448
52.205530571237205
52.2438044206296
52.282078270022005
52.320352119414416
52.35862596880681
52.396899818199216
52.43517366759162
52.47344751698402
52.51172136637642
52.54999521576883
52.58826906516123
52.62654291455363
52.664816763946035
52.70309061333843
52.74136446273084
52.77963831212325
52.81791216151564
52.85618601090805
52.894459860300444
52.93273370969285
52.97100755908526
53.00928140847766
53.04755525787006
53.08582910726246
53.12410295665486
53.16237680604728
53.2006506554396

68.43364271361591
68.47191656300834
68.51019041240073
68.54846426179313
68.58673811118553
68.62501196057794
68.66328580997035
68.70155965936274
68.73983350875514
68.77810735814755
68.81638120753995
68.85465505693234
68.89292890632476
68.93120275571715
68.96947660510956
69.00775045450196
69.04602430389436
69.08429815328677
69.12257200267918
69.16084585207157
69.19911970146397
69.23739355085637
69.27566740024878
69.31394124964119
69.35221509903359
69.39048894842598
69.4287627978184
69.46703664721079
69.5053104966032
69.5435843459956
69.58185819538801
69.6201320447804
69.6584058941728
69.6966797435652
69.73495359295761
69.77322744235002
69.81150129174242
69.84977514113481
69.88804899052722
69.92632283991964
69.96459668931203
70.00287053870443
70.04114438809682
70.07941823748924
70.11769208688163
70.15596593627404
70.19423978566644
70.23251363505885
70.27078748445125
70.30906133384364
70.34733518323605
70.38560903262847
70.42388288202086
70.46215673141326
70.50043058080566
70.5387044301980

86.03961343412114
86.07788728351355
86.11616113290594
86.15443498229834
86.19270883169075
86.23098268108315
86.26925653047556
86.30753037986796
86.34580422926035
86.38407807865276
86.42235192804516
86.46062577743756
86.49889962682997
86.53717347622238
86.57544732561477
86.61372117500717
86.65199502439958
86.690268873792
86.72854272318439
86.76681657257679
86.80509042196918
86.8433642713616
86.88163812075399
86.9199119701464
86.9581858195388
86.99645966893121
87.0347335183236
87.073007367716
87.11128121710841
87.14955506650082
87.18782891589322
87.22610276528562
87.26437661467801
87.30265046407042
87.34092431346284
87.37919816285523
87.41747201224763
87.45574586164004
87.49401971103244
87.53229356042483
87.57056740981724
87.60884125920964
87.64711510860205
87.68538895799445
87.72366280738684
87.76193665677926
87.80021050617167
87.83848435556406
87.87675820495646
87.91503205434886
87.95330590374127
87.99157975313368
88.02985360252607
88.06812745191847
88.10640130131088
88.14467515070328


<div style="border: 4px solid black;
            border-radius: 20px; color: white; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>3.5 Feature engenering for temporal variables</h1></center>
</div>

<p style="font-size: 18px; text-align: justify; line-height: 24px; font-family: Courier New;">As this is data related to trips and taxis, we identified that an analysis from the perspective of time spent on trips and their daily, monthly and annual occurrences could give us a holistic view of the data. Therefore, this specific data can expand the context to better understand its behavior, as well as identify outliers and focus on the dataset that could bring more value to analysis and decision making.
</br>

In [125]:
if not saved:
    data["year"] = data["time1"].str[0:4]
    data["month"] = data["time1"].str[5:7]
    data["hour"] = data["time1"].str[11:13]
    data["trip_pickup_date"] = pd.to_datetime(data["time1"]).dt.date
    data['trip_pickup_time'] = pd.to_datetime(data["time1"]).dt.time
    data["trip_dur_min"] = pd.to_datetime(data["trip_duration"], unit="s").dt.minute

<div style="border: 4px solid black;
            border-radius: 20px; color: white; background-attachment: fixed; color: black; padding: 5px;">
    <center><h1>3.6 Save Data</h1></center>
</div>

In [126]:
# Save the data to csv files, this way the process does not need to be repeated

cols = ["n_pass", "trip_duration", "area1", "area2", "year", "month", "hour"]

if not saved:
    data_clean = data[cols]
    data_analysis = data
    data_clean.to_csv("data/csv/data_clean.csv", index=False)
    data_analysis.to_csv("data/csv/data_analysis.csv", index=False)

In [127]:
# Final data clean
data_clean_final = pd.read_csv("data/csv/data_clean.csv")
data_clean_final

,n_pass,trip_duration,area1,area2,year,month,hour
0,1,405,79,64,2016,1,20
1,1,757,64,64,2016,3,3
2,2,1060,94,125,2016,3,22
3,1,304,48,32,2016,3,22
4,2,799,156,109,2016,1,18
...,...,...,...,...,...,...,...
261270,1,982,95,140,2016,1,1
261271,5,417,125,126,2016,4,13
261272,1,500,127,143,2016,5,11
261273,6,984,95,92,2016,6,12


In [128]:
# Final data analysis
data_analysis_final = pd.read_csv("data/csv/data_analysis.csv")
data_analysis_final

,index,id,vendor_id,time1,time2,n_pass,y1,x1,y2,x2,...,trip_duration,dist,area1,area2,year,month,hour,trip_pickup_date,trip_pickup_time,trip_dur_min
0,1389059,id0984631,2,2016-01-01 20:32:40,2016-01-01 20:39:25,1,-73.987457,40.728981,-73.988541,40.719597,...,405,0.009446,79,64,2016,1,20,2016-01-01,20:32:40,6
1,846163,id2158882,2,2016-03-06 03:42:56,2016-03-06 03:55:33,1,-73.991661,40.726768,-73.991966,40.722000,...,757,0.004778,64,64,2016,3,3,2016-03-06,03:42:56,12
2,1098897,id3078927,1,2016-03-18 22:24:47,2016-03-18 22:42:27,2,-73.991417,40.742203,-73.983604,40.764332,...,1060,0.023468,94,125,2016,3,22,2016-03-18,22:24:47,17
3,1349608,id3580013,2,2016-03-23 22:49:54,2016-03-23 22:54:58,1,-73.999756,40.714008,-74.006424,40.708183,...,304,0.008854,48,32,2016,3,22,2016-03-23,22:49:54,5
4,831079,id0568067,2,2016-01-18 18:13:12,2016-01-18 18:26:31,2,-73.973000,40.785309,-73.993279,40.752232,...,799,0.038799,156,109,2016,1,18,2016-01-18,18:13:12,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261270,1419373,id2692671,1,2016-01-23 01:16:35,2016-01-23 01:32:57,1,-73.983719,40.738880,-73.980141,40.770634,...,982,0.031955,95,140,2016,1,1,2016-01-23,01:16:35,16
261271,285629,id1014518,2,2016-04-01 13:53:01,2016-04-01 13:59:58,5,-73.977638,40.764450,-73.972153,40.763699,...,417,0.005537,125,126,2016,4,13,2016-04-01,13:53:01,6
261272,231464,id3003619,1,2016-05-13 11:38:29,2016-05-13 11:46:49,1,-73.957024,40.766327,-73.950356,40.776497,...,500,0.012161,127,143,2016,5,11,2016-05-13,11:38:29,8
261273,11462,id2251425,2,2016-06-07 12:04:58,2016-06-07 12:21:22,6,-73.982697,40.745232,-74.006203,40.743816,...,984,0.023549,95,92,2016,6,12,2016-06-07,12:04:58,16
